In [ ]:
!pip install --upgrade gensim

import pandas as pd
import numpy as np
import re, random, nltk, os
from gensim.models import Word2Vec

from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')

from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

nltk.download("stopwords")

SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

You should consider upgrading via the 'C:\Users\rafmarin\PycharmProjects\FHC_MLE_Flavor\venv\Scripts\python.exe -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'pandas'

Each row is a unique product. There are two columns, a de-identified company ID number and a flavor field. The flavor field may have multiple values in it that are comma separated (at first glance) so you may have to normalize it. Needs some scrubbing. We want to try to do a basic descriptive analysis first. See what is there. No one has ever looked. Number of uniques? Any common ones (value_counts)? Then we can think about embedding them and doing some clustering.

In [ ]:
df = pd.read_csv(r'C:\Users\Solliu\Documents\Deloitte\Projects\FHC\FlavorData.csv')

In [ ]:
df

In [ ]:
#check out the unique values for characterizing flavor. It seems like there are over 10,000 unique values for flavors.
print(df['Characterizing_Flavor'].unique())
print(df.Characterizing_Flavor.nunique())

In [ ]:
#Let's look at the top 10 most common flavors
df['Characterizing_Flavor'].value_counts().head(10)

In [ ]:
#Let's work on cleaning up the data. We see here a bunch of null values..
df[df['Characterizing_Flavor'].isnull()]

In [ ]:
#Let's now do some scrubbing
#words = {'vream':'cream', 'mallow':'marshmallow', 'rocksatar':'rockstar', 'crakcer':'cracker','starwberry':'strawberry',
#         'srawberry':'strawberry ','cockt':'cocktail','cocunut':'coconut','lycehee':'lychee','cocunut':'coconut',
#         'hawaiin':'hawaiian', 'forsted':'frosted','red, whiteandbiue':'red white and blue',
#        'banana mentho':'banana menthol', 'craker':'cracker',
#        'lrishcrea':'irish cream','cream cheese icin':'cream cheese icing'}


df['Characterizing_Flavor'].fillna(value='Not Found', inplace= True) #sub null values
df['Characterizing_Flavor'] = df['Characterizing_Flavor'].str.strip() #remove trailing and leading spaces
df['Characterizing_Flavor'] = df['Characterizing_Flavor'].str.replace(r"(?<=\w)-(?=\w)", " ")  # Replace dash between words
df['Characterizing_Flavor'] = df['Characterizing_Flavor'].str.lower() #normalize case
#df['Characterizing_Flavor'] = df['Characterizing_Flavor'].replace(words, regex=True) #fixing some typos
df['Characterizing_Flavor'] = df['Characterizing_Flavor'].str.replace(',','')
df['Characterizing_Flavor'] = df['Characterizing_Flavor'].str.replace('|','')
df['Characterizing_Flavor'] = df['Characterizing_Flavor'].str.replace('-','')
df['Characterizing_Flavor'] = df['Characterizing_Flavor'].str.replace(':','')
df['Characterizing_Flavor'] = df['Characterizing_Flavor'].str.replace(r'\s*[/mgl\d\s.]+$', '', regex=True) #removes trailing mg,numbers, characters
df['Characterizing_Flavor'] = df['Characterizing_Flavor'].str.replace(r"\s+", " ") #remove multiple spaces



In [ ]:
#Fix typos
replacements = {'mentho':'menthol','cocunut':'coconut','lycehee':'lychee','starwberry':'strawberry', 'srawberry':'strawberry',
                'strawberrry':'strawberry', 'stawberry':'strawberry','koconut':'coconut','rocksatar':'rockstar',
                'lrishcrea':'irish cream', 'icin':'icing', 'cockt':'cocktail','cocktai':'cocktail','coolmint':'cool mint',
                'vream':'cream', 'crea':'cream', 'mallow':'marshmallow','crakcer':'cracker', 'orangey':'orange',
                'whiteandbiue':'white and blue', 'watermelond':'watermelon','smoothi':'smoothie','coffe':'coffee',
                'cherryb1osso':'cherryblosso','caramel apples':'caramel apple','bllueberry':'blueberry',
                'banapanutbread':'banananutbread', 'strawberry and menthol':'strawberry menthol','menthol cotton candy':'cotton candy menthol',
                'grizzly apple coo':'grizzly apple','grape fruit':'grapefruit','cirque du salt berries 20mg/ml 2x':'cirque du salt berries',
                'cirque du salt berries 50mg/ml 2x':'cirque du salt berries', 'watermelon melon menthol':'watermelon menthol',
                'cinnamon roll (la) diy':'cinnamon roll (la)','cinnabunns':'cinnabun','alaska elixirs loaffetti formerly funfetti':'funfetti'
               }

def replace_all(text, dic):
    for i, j in dic.items():
        text = re.sub(r"\b%s\b" % i, j, text) # r"\b%s\b"% enables replacing by whole word matches only
    return text

df['Characterizing_Flavor'] = df['Characterizing_Flavor'].apply(lambda x: replace_all(x, replacements))

In [ ]:
#It looks like it changed # uniques and top 10! Nice. Let's keep going.
print(df.Characterizing_Flavor.nunique())
df['Characterizing_Flavor'].value_counts().head(10)

In [ ]:
#Taking a look at the least common flavors to see if theres anything else we need to scrub
df['Characterizing_Flavor'].value_counts(ascending=True).head(59)


In [ ]:
df[df['Characterizing_Flavor'].astype(str).str.contains('veno', regex = True)]


In [ ]:
df_emb[df_emb['Characterizing_Flavor'].astype(str).str.contains('venom', regex = True)]


In [ ]:
print(df.Company_ID.nunique())


Generate texts from tokens in a column using the tokenizer function, which tokenizes text as well as removes stopwords


In [ ]:
def tokenize_text(text, tokenizer, stopwords):
    tokens = tokenizer(text) # Get tokens from text
    tokens = [t for t in tokens if not t in stopwords]  # Remove stopwords
    tokens = [t.replace("'s","") for t in tokens] # remove 's
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    return tokens

In [ ]:
#generate tokens from text in tokens column

custom_stopwords = set(stopwords.words("english"))
text_columns = ["Characterizing_Flavor"]

df_emb = df.copy()
df_emb.insert(2,"tokens",0)

#convert to string
for col in text_columns:
    df_emb[col] = df_emb[col].astype(str)

# tokenize text
df_emb["tokens"] = df_emb["Characterizing_Flavor"].map(lambda x: tokenize_text(x, word_tokenize, custom_stopwords))
print(df_emb)

# Remove duplicated after preprocessing
df_emb_uniq = df_emb.copy()
_, idx = np.unique(df_emb_uniq["tokens"], return_index=True)
df_emb_uniq = df_emb_uniq.iloc[idx, :]

# Remove empty values and keep relevant columns
df_emb = df_emb.loc[df_emb.tokens.map(lambda x: len(x) > 0), ["Characterizing_Flavor","tokens"]]
df_emb_uniq = df_emb_uniq.loc[df_emb_uniq.tokens.map(lambda x: len(x) > 0), ["Characterizing_Flavor","tokens"]]


print(f"Original dataframe: {df.shape}")
print(f"Pre-processed dataframe, w duplicates: {df_emb.shape}")
print(f"Pre-processed dataframe, without duplicates: {df_emb_uniq.shape}")




In [ ]:
print(df_emb_uniq)

Now we will train a Word2Vec model using the tokens we generated earlier. This will output an array of vectors or number values for each token

In [ ]:
tokenized_docs = df_emb.tokens.tolist()

model = Word2Vec(sentences=tokenized_docs, vector_size=100, workers=1, seed=SEED)

def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
        list_of_docs: List of documents
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features

vectorized_docs = vectorize(tokenized_docs, model=model)

## what do these represent? length of the list of documents, size of the generated vectors
len(vectorized_docs), len(vectorized_docs[0])

How many of the tokenized words are making it into the trained model?

In [ ]:
#check # words in the resulting tokenized words.
flat_list = [item for sublist in tokenized_docs for item in sublist] #turn array of arrays into 1 array
print(len(flat_list))
unique_list = []
[unique_list.append(x) for x in flat_list if x not in unique_list] #remove duplicates
print("num unique words: ", len(unique_list))


In [ ]:
model.wv.key_to_index["flavor"]


In [ ]:
model.wv.key_to_index["strawberry"]

In [ ]:
#get a list of vocab for the trained model
#why are we getting less words for trained model?
trained = []
for item in model.wv.index_to_key:
    trained.append(item)

print("num of words in trained model: ", len(trained))

unique_trained = []

[unique_trained.append(x) for x in trained if x not in unique_trained] #remove duplicates
print("num unique words: ", len(unique_trained))

In [ ]:
model.wv.similarity('banana', 'menthol')

In [ ]:
model.wv.most_similar("fruit")

Implement PCA to look at the variance of the data

In [2]:
np.shape(vectorized_docs) #9390 examples with 100 variables for each example


NameError: name 'np' is not defined

In [3]:
def PCA(X , num_components):
    #center the data by subtracting the mean
    X_meaned = X - np.mean(X , axis = 0)

    # calculating the covariance matrix of the mean-centered data.
    cov_mat = np.cov(X_meaned , rowvar = False)

    #Calculating Eigenvalues and Eigenvectors of the covariance matrix. A higher Eigenvalue corresponds to a higher variability.
    #each column in the Eigen vector-matrix corresponds to a principal component
    eigen_values , eigen_vectors = np.linalg.eigh(cov_mat)

    #sort the eigenvalues in descending order. arranging them in descending order of their Eigenvalue will automatically
    #arrange the principal component in descending order of their variability.Hence the first column in our rearranged
    #Eigen vector-matrix will be a principal component that captures the highest variability
    sorted_index = np.argsort(eigen_values)[::-1]
    sorted_eigenvalue = eigen_values[sorted_index]
    sorted_eigenvectors = eigen_vectors[:,sorted_index]#similarly sort the eigenvectors

    #take top values depending on # components
    eigenvector_subset = sorted_eigenvectors[:,0:num_components]

    #reduces dimension from 9390, 100 to 9390, 2
    X_reduced = np.dot(eigenvector_subset.transpose() , X_meaned.transpose() ).transpose()

    print(X_reduced)
    return X_reduced

In [4]:
mat_reduced = PCA(df_emb , 2)

#Creating a Pandas DataFrame of reduced Dataset
principal_df = pd.DataFrame(mat_reduced , columns = ['PC1','PC2'])

#Concat it with target variable to create a complete Dataset
principal_df = pd.concat([principal_df , df_emb['Characterizing_Flavor']] , axis = 1)

principal_df



NameError: name 'df_emb' is not defined

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt

plt.figure(figsize = (6,6))
sb.scatterplot(data = principal_df , x = 'PC1',y = 'PC2' , hue = 'Characterizing_Flavor' , s = 60 , palette= 'icefire')


Let me try applying PCA to the original dataframe? That would make sense for being able to see all the different characterizing flavors...Can I vectorize without losing all the data first?

In [ ]:
## calculating optimal value of k using elbow method
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt
import os



In [ ]:
# Visualizing the data
plt.plot()
plt.xlim([0, .4])
plt.ylim([0, .4])
plt.title('Dataset')
plt.scatter(vectorized_docs[0],vectorized_docs[1])
plt.show()


Calculating values of Distortion and Intertia to find the optimal K value

In [ ]:
#Building the clustering model and calculating the values of the Distortion and Inertia
distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
K = range(1, 40)
X = np.array(list(zip(vectorized_docs[0],vectorized_docs[1]))).reshape(len(vectorized_docs[0]), 2) #what does this X mean?
os.environ["OMP_NUM_THREADS"] = "1"

for k in K:
    # Building and fitting the model
    kmeanModel = KMeans(n_clusters=k).fit(X)
    kmeanModel.fit(X)

    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                        'euclidean'), axis=1)) / X.shape[0])
    inertias.append(kmeanModel.inertia_)

    mapping1[k] = sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                   'euclidean'), axis=1)) / X.shape[0]
    mapping2[k] = kmeanModel.inertia_

In [ ]:
#Using the different values of Distortion
for key, val in mapping1.items():
	print(f'{key} : {val}')

plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('The Elbow Method using Distortion')
plt.show()

In [ ]:
for key, val in mapping2.items():
    print(f'{key} : {val}')

plt.plot(K, inertias, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Inertia')
plt.title('The Elbow Method using Inertia')
plt.show()

To determine the optimal number of clusters, we have to select the value of k at the “elbow” ie the point after which the distortion/inertia start decreasing in a linear fashion and is not rapidly changing anymore. Based on these graphs, the **optimal K value looks to be 5 (inertia) or 9 (distortion)**

In [ ]:
def mbkmeans_clusters(
	X,
    k,
    mb,
    print_silhouette_values,
):
    """Generate clusters and print Silhouette metrics using MBKmeans

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

In [ ]:
#higher sil. coef is denser clusters
#k is # clusters. look into intracluster distance(avg distance of all the points from the center), graph & elbow point
#mini k vs k

os.environ["OMP_NUM_THREADS"] = "2"

clustering, cluster_labels = mbkmeans_clusters(
	X=vectorized_docs,
    k=4,
    mb=500,
    print_silhouette_values=True,
)
df_clusters = pd.DataFrame({
    "text": vectorized_docs,
    "tokens": [" ".join(text) for text in tokenized_docs],
    "cluster": cluster_labels
})

In [ ]:
print("Most representative terms per cluster (based on centroids):")
for i in range(4):
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=10)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")

When k=5, Clusters 1 & 4 have almost exact top 10 representative words. 4 clusters seems to be the best in this case

In [556]:

#to do
#clean data more
#embedding failures. how much of the vocab is being represented?
--not all words are in word2vec. are there words that are not being represented in there?
--compare our vocab vs word2vec vocab. how much % of vocab is being represented in that vocab?

#plot the clusters, pca to check variance of clusters, want to keep 70-80% of data, t-sne
--pca: dimensionality reduction (e.g. groups columns into principal components)
--chunking flavors that are mixed together
--check dimensionality reduction is adequate
--use 2 or 3 of vectors representing each number (right now there are 100 per word)
--pca also used to determine how many cluster for k means

#plot the clusters, coloring in by company ID

#less important but also to do
#k means vs mini k means

In [ ]:
df_emb[df_emb['Characterizing_Flavor'].astype(str).str.contains('banana', regex = True)]

In [ ]:
df[df['Characterizing_Flavor'].astype(str).str.contains('icin', regex = True)]

In [5]:
df[df['Characterizing_Flavor'].astype(str).str.contains('zombie', regex = True)]

NameError: name 'df' is not defined

fu
